# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [2]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug
!pip install protobuf==3.20.0
!pip install --upgrade bokeh==2.4.3

  Obtaining dependency information for smdebug from https://files.pythonhosted.org/packages/84/6a/d1d35ecbddfadbe1040fa69aaca23665f3ad2c16c519571855589721dfec/smdebug-1.0.34-py2.py3-none-any.whl.metadata
  Using cached smdebug-1.0.34-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
  Using cached pyinstrument_cext-0.2.4-cp310-cp310-linux_x86_64.whl
Using cached smdebug-1.0.34-py2.py3-none-any.whl (280 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.4
    Uninstalling protobuf-4.24.4:
      Successfully uninstalled protobuf-4.24.4

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached protobuf-3.20.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation

In [2]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
from sagemaker import get_execution_role
import boto3
import os
import subprocess
import torch


from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

from sagemaker.debugger import (
Rule, ProfilerRule, rule_configs, DebuggerHookConfig, ProfilerConfig, FrameworkProfile
)

from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts
import IPython
from sagemaker.predictor import Predictor
from sagemaker.serializers import IdentitySerializer
import base64


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
[2023-10-25 10:35:13.804 sagemaker-data-scienc-ml-t3-medium-26b76b1d598bfaa012ec45c79dc5:68 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None


In [3]:
# Setup Sagemaker Session 
sagemaker_session = sagemaker.Session(default_bucket='dog-classifier')
bucket = sagemaker_session.default_bucket()
execution_role = sagemaker.session.get_execution_role()
region = sagemaker_session.boto_region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [4]:
#define data location constants
local_data_dir = 'data'  
s3_data_path = f's3://{bucket}/data'
s3_model_path = f's3://{bucket}/model'

In [6]:
# Command to download and unzip data
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
subprocess.run(f"unzip dogImages.zip -d {local_data_dir}", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
os.remove('dogImages.zip')

subprocess.run(f"aws s3 sync ./{local_data_dir} {s3_data_path}", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

--2023-10-24 21:30:28--  https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.194.40, 52.219.112.224, 52.219.116.64, ...
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.194.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1132023110 (1.1G) [application/zip]
Saving to: ‘dogImages.zip’

dogImages.zip       100%[===================>]   1.05G  52.5MB/s    in 33s     

2023-10-24 21:31:04 (32.3 MB/s) - ‘dogImages.zip’ saved [1132023110/1132023110]



CompletedProcess(args='aws s3 sync ./data s3://dog-classifier/data', returncode=0)

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [7]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "learningRate": ContinuousParameter(0.001, 0.1),
    "batch-size": CategoricalParameter([32, 64, 128, 256, 512]),
    "epochs": IntegerParameter(2, 4)
}

In [8]:
#TODO: Create estimators for your HPs

estimator = PyTorch(
    entry_point="hpo.py",
    role=execution_role,
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="1.8",
    py_version="py36",
)

objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "average test loss: ([0-9\\.]+)"}]

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=2, 
    objective_type=objective_type,
)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [8]:
data_channels = {
    'train': f'{s3_data_path}/dogImages/train',  # Replace with your S3 path for training data
    'validation': f'{s3_data_path}/dogImages/valid', # Replace with your S3 path for validation data
    'test': f'{s3_data_path}/dogImages/test' 
}

In [ ]:
#Fit your HP Tuner

tuner.fit(wait=True, inputs=data_channels, include_cls_metadata=False) # TODO: Remember to include your data channels

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


Using provided s3_resource
.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [27]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()


2023-10-19 16:52:39 Starting - Preparing the instances for training
2023-10-19 16:52:39 Downloading - Downloading input data
2023-10-19 16:52:39 Training - Training image download completed. Training in progress.
2023-10-19 16:52:39 Uploading - Uploading generated training model
2023-10-19 16:52:39 Completed - Resource reused by training job: pytorch-training-231019-1609-004-fb3c2952


{'_tuning_objective_metric': '"average test loss"',
 'batch-size': '"512"',
 'epochs': '4',
 'learningRate': '0.03227665063315492',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"pytorch-training-2023-10-19-16-09-04-360"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-508063414633/pytorch-training-2023-10-19-16-09-04-360/source/sourcedir.tar.gz"'}

In [5]:
#load best results on kernal restart
tuning_job_name = "pytorch-training-231024-2137"
tuning_job_description = sagemaker_session.sagemaker_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

print("Tuning job name:", tuning_job_description['HyperParameterTuningJobName'])
print("Tuning job status:", tuning_job_description['HyperParameterTuningJobStatus'])
print("Best training job name:", tuning_job_description['BestTrainingJob']['TrainingJobName'])

best_hyperparameters = tuning_job_description['BestTrainingJob']['TunedHyperParameters']
best_hyperparameters = {
    'batch-size': int(best_hyperparameters['batch-size'].strip('"')),
    'epochs': int(best_hyperparameters['epochs']),
    'learningRate': float(best_hyperparameters['learningRate'])
}
print("Best hyperparameters:", best_hyperparameters)

#Get the hyperparameters of the best trained model
# best_hyper_parameters = best_estimator.hyperparameters()

Tuning job name: pytorch-training-231024-2137
Tuning job status: Completed
Best training job name: pytorch-training-231024-2137-004-cb6919ee
Best hyperparameters: {'batch-size': 512, 'epochs': 3, 'learningRate': 0.0027853603934280147}


## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [6]:
# TODO: Set up debugging and profiling rules and hooks
#Debug Configs
rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)

debugger_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "100", "eval.save_interval": "10"}
)



Framework profiling will be deprecated from tensorflow 2.12 and pytorch 2.0 in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [14]:
# Create and fit an estimator

estimator = PyTorch(
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    entry_point="train_model.py",
    framework_version="1.8",
    py_version="py36",
    output_path=s3_model_path,
    hyperparameters=best_hyperparameters,
    profiler_config=profiler_config,
    debugger_hook_config=debugger_config,
    rules=rules,
)

estimator.fit(wait=True, inputs=data_channels)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker:Creating training-job with name: pytorch-training-2023-10-25-11-04-38-486


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTrainingJob operation: The account-level service limit 'ml.p3.2xlarge for training job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.

In [31]:
# TODO: Plot a debugging output.
training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

Training jobname: pytorch-training-2023-10-20-20-45-00-654
Region: us-east-1


In [32]:
trial = create_trial(estimator.latest_job_debugger_artifacts_path())
print(trial.tensor_names())
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.TRAIN)))
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.EVAL)))

[2023-10-20 21:08:53.768 ip-172-16-168-194.ec2.internal:4777 INFO s3_trial.py:42] Loading trial debug-output at path s3://sagemaker-us-east-1-508063414633/pytorch-training-2023-10-20-20-45-00-654/debug-output
[2023-10-20 21:08:54.128 ip-172-16-168-194.ec2.internal:4777 INFO trial.py:197] Training has ended, will refresh one final time in 1 sec.
[2023-10-20 21:08:55.147 ip-172-16-168-194.ec2.internal:4777 INFO trial.py:210] Loaded all steps
['CrossEntropyLoss_output_0', 'gradient/ResNet_fc.0.bias', 'gradient/ResNet_fc.0.weight', 'layer1.0.relu_input_0', 'layer1.0.relu_input_1', 'layer1.1.relu_input_0', 'layer1.1.relu_input_1', 'layer2.0.relu_input_0', 'layer2.0.relu_input_1', 'layer2.1.relu_input_0', 'layer2.1.relu_input_1', 'layer3.0.relu_input_0', 'layer3.0.relu_input_1', 'layer3.1.relu_input_0', 'layer3.1.relu_input_1', 'layer4.0.relu_input_0', 'layer4.0.relu_input_1', 'layer4.1.relu_input_0', 'layer4.1.relu_input_1', 'relu_input_0']
1
1


In [33]:
tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

! aws s3 ls {rule_output_path} --recursive
! aws s3 cp {rule_output_path} ./ --recursive


ProfilerConfig:{'S3OutputPath': 's3://sagemaker-us-east-1-508063414633/', 'ProfilingIntervalInMilliseconds': 500, 'ProfilingParameters': {'DataloaderProfilingConfig': '{"StartStep": 0, "NumSteps": 10, "MetricsRegex": ".*", }', 'DetailedProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }', 'FileOpenFailThreshold': '50', 'HorovodProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }', 'LocalPath': '/opt/ml/output/profiler', 'PythonProfilingConfig': '{"StartStep": 0, "NumSteps": 10, "ProfilerName": "cprofile", "cProfileTimer": "total_time", }', 'RotateFileCloseIntervalInSeconds': '60', 'RotateMaxFileSizeInBytes': '10485760', 'SMDataParallelProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }'}, 'DisableProfiler': False}
s3 path:s3://sagemaker-us-east-1-508063414633/pytorch-training-2023-10-20-20-45-00-654/profiler-output


Profiler data from system is available
[2023-10-20 21:09:00.270 ip-172-16-168-194.ec2.internal:4777 INFO metrics_reader_base.py:134] Getting 16 event files
select eve

You will find the profiler report in s3://sagemaker-us-east-1-508063414633/pytorch-training-2023-10-20-20-45-00-654/rule-output
2023-10-20 21:01:46     417364 pytorch-training-2023-10-20-20-45-00-654/rule-output/ProfilerReport/profiler-output/profiler-report.html
2023-10-20 21:01:46     274027 pytorch-training-2023-10-20-20-45-00-654/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb
2023-10-20 21:01:41        536 pytorch-training-2023-10-20-20-45-00-654/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json
2023-10-20 21:01:41      35424 pytorch-training-2023-10-20-20-45-00-654/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
2023-10-20 21:01:41       1900 pytorch-training-2023-10-20-20-45-00-654/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
2023-10-20 21:01:41        310 pytorch-training-2023-10-20-20-45-00-654/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.jso

In [35]:
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")

## Report Analysis 

After looking over the report I think once change I can make is to increase the batch size again and retrain the model one last time. 

In [37]:
print(best_hyperparameters)

{'batch-size': 512, 'epochs': 4, 'learningRate': 0.03227665063315492, 'sagemaker_submit_directory': 's3://sagemaker-us-east-1-508063414633/pytorch-training-2023-10-20-20-45-00-654/source/sourcedir.tar.gz', 'sagemaker_program': 'train_model.py', 'sagemaker_container_log_level': 20, 'sagemaker_job_name': 'pytorch-training-2023-10-20-20-45-00-654', 'sagemaker_region': 'us-east-1'}


In [38]:
best_hyperparameters['batch-size'] = 1024
print(best_hyperparameters)

{'batch-size': 1024, 'epochs': 4, 'learningRate': 0.03227665063315492, 'sagemaker_submit_directory': 's3://sagemaker-us-east-1-508063414633/pytorch-training-2023-10-20-20-45-00-654/source/sourcedir.tar.gz', 'sagemaker_program': 'train_model.py', 'sagemaker_container_log_level': 20, 'sagemaker_job_name': 'pytorch-training-2023-10-20-20-45-00-654', 'sagemaker_region': 'us-east-1'}


In [36]:
estimator = PyTorch(
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    entry_point="train_model.py",
    framework_version="1.8",
    py_version="py36",
    hyperparameters=best_hyperparameters,
    profiler_config=profiler_config,
    debugger_hook_config=debugger_config,
    rules=rules,
)

estimator.fit(wait=True, inputs=data_channels)

NameError: name 'After' is not defined

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

deployment=estimator.deploy(
    initial_instance_count=1, 
    instance_type='ml.m5.xlarge'
) 

endpoint = deployment.endpoint_name
print(endpoint)

In [ ]:
predictor = Predictor(endpoint_name=endpoint)
predictor.serializer = IdentitySerializer("image/png")

In [ ]:

with open("./data/dogImages/test/014.Basenji/Basenji_00955.jpg", "rb") as f:
    image = f.read()
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()